# بناء وكلاء ذكاء اصطناعي بذاكرة مستدامة باستخدام Cognee

يوضح هذا الدفتر كيفية بناء وكلاء ذكاء اصطناعي ذكيين بقدرات ذاكرة متقدمة باستخدام [**cognee**](https://www.cognee.ai/) - ذاكرة ذكاء اصطناعي مفتوحة المصدر تجمع بين الرسوم البيانية للمعرفة، البحث الدلالي، وإدارة الجلسات لإنشاء أنظمة ذكاء اصطناعي مدركة للسياق.

## 🎯 أهداف التعلم

بنهاية هذا الدرس، ستفهم كيفية:
- **بناء رسوم بيانية للمعرفة مدعومة بالتضمينات**: تحويل النصوص غير المنظمة إلى معرفة منظمة وقابلة للاستعلام
- **تنفيذ ذاكرة الجلسة**: إنشاء محادثات متعددة الأدوار مع الاحتفاظ بالسياق تلقائيًا
- **تخزين المحادثات**: حفظ التفاعلات المهمة اختياريًا في ذاكرة طويلة الأمد للرجوع إليها في المستقبل
- **الاستعلام باستخدام اللغة الطبيعية**: الوصول إلى السياق التاريخي والاستفادة منه في المحادثات الجديدة
- **تصور الذاكرة**: استكشاف العلاقات في الرسم البياني للمعرفة الخاص بالوكيل


## 🏗️ ما الذي ستقوم ببنائه

في هذا الدليل، سنقوم بإنشاء **مساعد برمجي** بذاكرة مستدامة يقوم بما يلي:

### 1. **بناء قاعدة المعرفة**
   - يستوعب معلومات الملف الشخصي للمطور وخبراته
   - يعالج مبادئ البرمجة بلغة Python وأفضل الممارسات
   - يخزن المحادثات السابقة بين المطورين والمساعدات الذكية

### 2. **محادثات مدركة للجلسة**
   - يحافظ على السياق عبر أسئلة متعددة في نفس الجلسة
   - يقوم تلقائيًا بتخزين كل زوج من الأسئلة/الإجابات لاسترجاعها بكفاءة
   - يقدم ردودًا متماسكة وسياقية بناءً على تاريخ المحادثة

### 3. **ذاكرة طويلة الأمد**
   - يحتفظ بالمحادثات المهمة في ذاكرة طويلة الأمد
   - يسترجع الذكريات ذات الصلة من قاعدة المعرفة والجلسات السابقة لتوجيه التفاعلات الجديدة
   - يبني قاعدة معرفة متنامية تتحسن مع مرور الوقت

### 4. **استرجاع ذكي للذاكرة**
   - يستخدم البحث الدلالي المدرك للرسوم البيانية للعثور على المعلومات ذات الصلة عبر جميع المعرفة المخزنة
   - يقوم بتصفية عمليات البحث حسب مجموعات البيانات الفرعية (معلومات المطور مقابل المبادئ)
   - يجمع بين مصادر بيانات متعددة لتقديم إجابات شاملة


## 📋 المتطلبات والإعداد

### متطلبات النظام

قبل البدء، تأكد من توفر ما يلي:

1. **بيئة Python**
   - Python 3.9 أو أعلى
   - بيئة افتراضية (موصى بها)

2. **ذاكرة Redis** (مطلوبة لإدارة الجلسات)
   - Redis محلي: `docker run -d -p 6379:6379 redis`
   - أو استخدم خدمة Redis مُدارة

3. **وصول إلى واجهة برمجة التطبيقات LLM**
   - مفتاح API لـ OpenAI أو مقدمي خدمات آخرين (راجع [التوثيق](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **إعداد قاعدة البيانات**
   - لا توجد إعدادات مطلوبة افتراضيًا. يستخدم Cognee قواعد بيانات تعتمد على الملفات (LanceDB و Kuzu)
   - اختياريًا، يمكنك إعداد Azure AI Search كمخزن متجه (راجع [التوثيق](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### إعداد البيئة

قم بإنشاء ملف `.env` في دليل المشروع الخاص بك يحتوي على المتغيرات التالية:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ فهم بنية ذاكرة Cognee

### كيفية عمل Cognee

توفر Cognee نظام ذاكرة متقدم يتجاوز التخزين البسيط للقيم والمفاتيح:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### المكونات الرئيسية:

1. **رسم المعرفة**: يخزن الكيانات والعلاقات والروابط الدلالية
2. **التضمينات الشعاعية**: يتيح البحث الدلالي عبر جميع المعلومات المخزنة
3. **ذاكرة الجلسة المؤقتة**: يحافظ على سياق المحادثة داخل الجلسات وعبرها
4. **مجموعات العقد**: تنظم البيانات في فئات منطقية لاسترجاعها بشكل مستهدف

### أنواع الذاكرة في هذا الدرس:

- **الذاكرة الدائمة**: تخزين طويل الأمد في رسم المعرفة
- **ذاكرة الجلسة**: سياق المحادثة المؤقت في ذاكرة Redis المؤقتة
- **الذاكرة الدلالية**: البحث القائم على التشابه الشعاعي عبر جميع البيانات


## 📦 تثبيت الحزم المطلوبة

قم بتثبيت Cognee مع دعم Redis لإدارة الجلسات:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 تهيئة البيئة وتحميل المكتبات

تأكد من:
1. تشغيل Redis (على سبيل المثال، عبر Docker: `docker run -d -p 6379:6379 redis`)
2. ضبط متغيرات البيئة قبل استيراد وحدات التخزين المؤقت
3. إذا لزم الأمر، أعد تشغيل النواة وقم بتشغيل الخلايا بالترتيب

ستقوم الخلية التالية بـ:
1. تحميل متغيرات البيئة من `.env`
2. إعداد Cognee باستخدام إعدادات LLM الخاصة بك
3. تمكين التخزين المؤقت لإدارة الجلسات
4. التحقق من اتصال جميع المكونات بشكل صحيح


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 إعداد أدلة التخزين

تستخدم Cognee دليلين منفصلين لعملياتها:
- **الجذر البيانات**: يخزن المستندات المُدخلة والبيانات المعالجة
- **الجذر النظام**: يحتوي على قاعدة بيانات الرسم البياني المعرفي وبيانات النظام الوصفية

سنقوم بإنشاء أدلة معزولة لهذا الدليل كما يلي:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 إعادة تعيين حالة الذاكرة

قبل أن نبدأ في بناء نظام الذاكرة الخاص بنا، دعونا نتأكد من أننا نبدأ من جديد.

> 💡 **نصيحة**: يمكنك تخطي هذه الخطوة إذا كنت ترغب في الاحتفاظ بالذكريات الحالية من تشغيلاتك السابقة عند استخدامك لهذا الدفتر لاحقًا.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 الجزء 1: بناء قاعدة المعرفة

### مصادر البيانات لمساعد المطور

سنقوم بجمع ثلاثة أنواع من البيانات لإنشاء قاعدة معرفة شاملة:

1. **ملف تعريف المطور**: الخبرة الشخصية والخلفية التقنية
2. **أفضل ممارسات Python**: فلسفة Python مع إرشادات عملية
3. **المحادثات التاريخية**: جلسات الأسئلة والأجوبة السابقة بين المطورين ومساعدي الذكاء الاصطناعي

تتيح هذه البيانات المتنوعة لوكيلنا أن:
- يفهم السياق التقني للمستخدم
- يطبق أفضل الممارسات في التوصيات
- يتعلم من التفاعلات الناجحة السابقة


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 تحويل البيانات إلى رسم بياني للمعرفة

سنقوم الآن بتحويل النص الخام إلى ذاكرة منظمة. تتضمن هذه العملية:

1. **إضافة البيانات إلى NodeSets**: تنظيم المعلومات في فئات منطقية
   - `developer_data`: ملف تعريف المطور والمحادثات
   - `principles_data`: أفضل ممارسات وإرشادات Python

2. **تشغيل خط أنابيب Cognify**: استخراج الكيانات والعلاقات وإنشاء التضمينات
   - تحديد المفاهيم الرئيسية
   - إنشاء روابط دلالية بين المعلومات ذات الصلة
   - توليد تضمينات متجهة

قد يستغرق هذا بعض الوقت بينما يقوم نموذج اللغة الكبير بمعالجة النص وبناء هيكل الرسم البياني:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 تصور الرسم البياني للمعرفة

لنستكشف هيكل الرسم البياني للمعرفة الخاص بنا. تعرض التصور ما يلي:
- **العُقد**: الكيانات المستخرجة من النص (المفاهيم، التقنيات، الأشخاص)
- **الحواف**: العلاقات والروابط بين الكيانات
- **المجموعات**: المفاهيم ذات الصلة المجمعة بناءً على التشابه الدلالي

افتح ملف HTML المُنشأ في متصفحك لاستكشاف الرسم البياني بشكل تفاعلي:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 إثراء الذاكرة باستخدام Memify

وظيفة `memify()` تقوم بتحليل مخطط المعرفة وتوليد قواعد ذكية حول البيانات. هذه العملية:
- تحدد الأنماط وأفضل الممارسات
- تنشئ إرشادات قابلة للتنفيذ بناءً على المحتوى
- تقيم العلاقات بين مجالات المعرفة المختلفة

تساعد هذه القواعد الوكيل على اتخاذ قرارات أكثر وعيًا عند الإجابة على الأسئلة. التقاط تصور ثانٍ يساعدك على مقارنة كيفية تكثيف المخطط بمجرد إثرائه.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 الجزء 2: استرجاع الذاكرة الذكي

### العرض التوضيحي 1: دمج المعرفة عبر المستندات

الآن بعد أن تم بناء الرسم البياني المعرفي، دعونا نختبر كيف يقوم Cognee بدمج المعلومات من مصادر متعددة للإجابة على الأسئلة المعقدة.

الاستعلام الأول يوضح:
- **الفهم الدلالي**: العثور على المفاهيم ذات الصلة حتى عندما لا يتم ذكرها صراحة
- **الربط المتقاطع**: دمج ملف تعريف المطور مع مبادئ Python
- **الاستدلال السياقي**: تطبيق أفضل الممارسات على تنفيذات محددة

### العرض التوضيحي 2: البحث المصفى باستخدام NodeSets

الاستعلام الثاني يوضح كيفية استهداف مجموعات فرعية محددة من الرسم البياني المعرفي:
- يستخدم معلمة `node_name` للبحث فقط داخل `principles_data`
- يقدم إجابات مركزة من مجال معرفي محدد
- مفيد عندما تحتاج إلى معلومات خاصة بمجال معين


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 الجزء 3: إعداد إدارة الجلسات

### تفعيل ذاكرة المحادثة

إدارة الجلسات أمر ضروري للحفاظ على السياق عبر عدة تفاعلات. هنا سنقوم بـ:

1. **تهيئة سياق المستخدم**: إنشاء أو استرجاع ملف تعريف المستخدم لتتبع الجلسة
2. **تكوين محرك التخزين المؤقت**: الاتصال بـ Redis لتخزين سجل المحادثات
3. **تفعيل متغيرات الجلسة**: إعداد متغيرات السياق التي تستمر عبر الاستفسارات

> ⚠️ **هام**: يتطلب هذا تشغيل Redis ووجود `CACHING=true` في بيئتك


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ وظيفة مساعدة: عرض سجل الجلسة

تتيح لنا هذه الوظيفة المساعدة فحص سجل المحادثات المخزن في Redis. وهي مفيدة لـ:
- تصحيح أخطاء إدارة الجلسات
- التحقق من أن المحادثات يتم تخزينها مؤقتًا
- فهم السياق المتاح للوكيل


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## الجلسة 1: مختبر دعم Async — السؤال الأول

ابدأ جلسة `async-support-lab` بسؤال عن أنماط asyncio المناسبة للتعامل مع التتبع التلقائي الضخم بطريقة تدعم التتبع عن بُعد (telemetry). الرسم البياني لديه بالفعل معرفة بـ asyncio، و aiohttp، وممارسات المراقبة، لذا يجب أن تعكس الإجابة المحادثات السابقة مع تخصيصها للسؤال الجديد.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## فحص ذاكرة الجلسة 1 بعد التبادل الأول

تشغيل `show_history(session_1)` مباشرة بعد السؤال الأول يؤكد أن Cognee قامت بكتابة كل من الطلب والإكمال في Redis. يجب أن ترى إدخالًا واحدًا مع إرشادات التزامن.


In [ ]:
await show_history(session_1)

## الجلسة 1: متابعة حول نماذج البيانات

بعد ذلك نسأل، "متى يجب أن أختار dataclasses بدلاً من Pydantic؟" باستخدام نفس معرف الجلسة. يجب على Cognee أن تجمع بين مبادئ Python والمحادثات السابقة حول FastAPI لتقديم نصيحة دقيقة—مظهرة أن السياق يستمر داخل جلسة مسماة.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## تأكيد أن سجل الجلسة 1 يحتوي على كلا الجانبين

يجب أن يكشف استدعاء آخر لـ `show_history(session_1)` عن وجود مدخلين للأسئلة والأجوبة. يتطابق هذا مع خطوة "إعادة تشغيل الذاكرة" في مختبر Mem0 ويثبت أن الأدوار الإضافية تُضاف إلى نفس النص.


In [ ]:
await show_history(session_1)

## الجلسة 2: سلسلة مراجعة التصميم — جلسة جديدة

لإظهار العزل بين السلاسل، نقوم بتشغيل `design-review-session` ونطلب إرشادات تسجيل لمراجعات الحوادث. على الرغم من أن قاعدة المعرفة الأساسية هي نفسها، إلا أن معرف الجلسة الجديد يحافظ على فصل النصوص.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## مراجعة الجلسة 2 التاريخ

`show_history(session_2)` يجب أن يعرض فقط زوج الطلب/الاستجابة الخاص بمراجعة التصميم. قارن ذلك مع الجلسة 1 لتوضيح كيف يحافظ Cognee على نصوص مستقلة بينما يعيد استخدام الرسم البياني للمعرفة المشتركة.


In [ ]:
await show_history(session_2)

## الملخص

تهانينا! لقد قمت للتو بإضافة طبقة ذاكرة طويلة المدى لمساعدك البرمجي مدعومة بواسطة Cognee.

في هذا الدليل، قمت بتحويل محتوى المطور الخام (كود، مستندات، محادثات) إلى ذاكرة تعتمد على الرسم البياني + المتجهات يمكن لوكيلك البحث فيها، التفكير من خلالها، وتحسينها باستمرار.

ما الذي تعلمته

1. **من النص الخام إلى ذاكرة الذكاء الاصطناعي**: كيفية قيام Cognee باستيعاب البيانات غير المهيكلة وتحويلها إلى ذاكرة ذكية وقابلة للبحث باستخدام بنية تجمع بين المتجهات + الرسم البياني المعرفي.

2. **إثراء الرسم البياني باستخدام memify**: كيفية تجاوز إنشاء الرسم البياني الأساسي واستخدام memify لإضافة حقائق مستنتجة وعلاقات أكثر ثراءً فوق الرسم البياني الحالي.

3. **استراتيجيات بحث متعددة**: كيفية استعلام الذاكرة باستخدام أنواع بحث مختلفة (أسئلة وأجوبة مدركة للرسم البياني، إكمال بأسلوب RAG، رؤى، أجزاء خام، بحث في الكود، إلخ) بناءً على احتياجات وكيلك.

4. **استكشاف بصري**: كيفية فحص وتصحيح ما أنشأته Cognee باستخدام تصورات الرسم البياني وواجهة Cognee، بحيث يمكنك رؤية كيفية هيكلة المعرفة فعليًا.

5. **ذاكرة مدركة للجلسة**: كيفية دمج سياق كل جلسة مع ذاكرة دلالية مستمرة بحيث يمكن للوكلاء التذكر عبر الجلسات دون تسريب المعلومات بين المستخدمين.


## النقاط الرئيسية
1. الذاكرة كرسوم بيانية معرفية مدعومة بالتضمينات

   - **فهم منظم**: تجمع Cognee بين مخزن متجهات ومخزن رسوم بيانية، مما يجعل بياناتك قابلة للبحث من حيث المعنى ومتصلة بالعلاقات. تستخدم Cognee قواعد بيانات تعتمد على الملفات بشكل افتراضي (LanceDB لمخزن المتجهات، وKuzu لمخزن الرسوم البيانية).

   - **استرجاع مدرك للعلاقات**: يمكن أن تكون الإجابات مستندة ليس فقط إلى "النصوص المشابهة"، ولكن أيضًا إلى كيفية ارتباط الكيانات ببعضها البعض.

   - **ذاكرة حية**: تتطور طبقة الذاكرة وتنمو وتظل قابلة للاستعلام كرسوم بيانية متصلة واحدة.

2. أوضاع البحث والاستدلال
   - **استرجاع هجين**: يمزج البحث بين تشابه المتجهات، وهيكل الرسوم البيانية، واستدلال LLM، بدءًا من البحث عن أجزاء خام وصولاً إلى الإجابة على الأسئلة المدركة للرسوم البيانية.

   - **تكييف الوضع مع المهمة**: استخدم أوضاعًا بأسلوب الإكمال عندما تحتاج إلى إجابات بلغة طبيعية، وأوضاع الأجزاء/الملخص/الرسوم البيانية عندما يحتاج وكيلك إلى سياق خام أو لتحفيز استدلاله الخاص.

3. وكلاء مخصصون ومدركون للجلسة
   - **سياق الجلسة + ذاكرة طويلة الأمد**: تحتفظ Cognee بسياق "الخيط" قصير الأمد منفصلًا عن الذاكرة طويلة الأمد على مستوى المستخدم أو المؤسسة.

## التطبيقات الواقعية

1. **وكلاء الذكاء الاصطناعي العمودية**

   استخدم النمط من هذا الدفتر لتشغيل مساعدات ذكية في مجالات محددة تعتمد على Cognee كنواة للاسترجاع والاستدلال:

- **مساعدات المطورين**: مراجعة الكود، تحليل الحوادث، ومساعدات التصميم التي تتنقل عبر الكود، واجهات برمجة التطبيقات، وثائق التصميم، والتذاكر كرسوم بيانية ذاكرية واحدة.

- **مساعدات موجهة للعملاء**: وكلاء دعم أو نجاح يستخرجون من وثائق المنتجات، الأسئلة الشائعة، ملاحظات إدارة علاقات العملاء، والتذاكر السابقة مع استرجاع مدرك للرسوم البيانية وإجابات مستشهد بها.

- **مساعدات الخبراء الداخلية**: مساعدات في السياسات، القانونية، أو الأمنية تستدل على القواعد المترابطة، الإرشادات، والقرارات التاريخية بدلاً من ملفات PDF المعزولة.

   تم تصميم Cognee بشكل صريح كذاكرة دقيقة ودائمة لوكلاء الذكاء الاصطناعي، مما يوفر رسمًا بيانيًا معرفيًا حيًا يمكن إدراجه خلف وكيلك ليحل محل التوليفات العشوائية من مخازن المتجهات وكود الرسوم البيانية المخصص.

2. **توحيد صوامع البيانات في ذاكرة واحدة**

   يساعد نفس النهج أيضًا في بناء طبقة ذاكرة موحدة عبر مصادر متفرقة:

- **من الصوامع إلى رسم بياني واحد**: استوعب البيانات المنظمة (مثل قواعد البيانات) وغير المنظمة (مثل المستندات، المحادثات) في رسم بياني واحد مدعوم بالتضمينات، بدلاً من فهارس منفصلة لكل نظام.

- **الاستدلال عبر المصادر مع الاستشهادات**: قم بتشغيل استدلال متعدد الخطوات على كل شيء—"اربط" السجلات، المقاييس، والمستندات عبر الرسم البياني—ولا تزال تقدم إجابات مستندة مع مصدرها.

- **مراكز المعرفة**: في مجالات مثل البنوك أو التعليم، تُستخدم Cognee بالفعل لتوحيد ملفات PDF، الأنظمة الداخلية، وبيانات التطبيقات في رسم بياني معرفي واحد مع متجهات بحيث يمكن للوكلاء الإجابة على الأسئلة بسياق دقيق ومستشهد به.

## الخطوات التالية

لقد قمت بتنفيذ الحلقة الأساسية للذاكرة. إليك بعض التوسعات الطبيعية التي يمكنك تجربتها بنفسك (راجع [وثائق Cognee](https://docs.cognee.ai/) للحصول على التفاصيل):

1. **التجربة مع الوعي الزمني**: قم بتفعيل "الوعي الزمني" لاستخراج الأحداث والطوابع الزمنية من النصوص.

2. **إدخال الاستدلال المدفوع بالأونتولوجيا**: قم بتعريف أونتولوجيا OWL لمجالك. استخدم دعم الأونتولوجيا في Cognee بحيث تكون الكيانات والعلاقات المستخرجة مستندة إلى هذا المخطط، مما يحسن جودة الرسم البياني والإجابات الخاصة بالمجال.

3. **إضافة حلقة تغذية راجعة**: دع Cognee تعدل أوزان حواف الرسم البياني بناءً على تعليقات المستخدمين الحقيقية، بحيث يتحسن الاسترجاع بمرور الوقت بدلاً من أن يظل ثابتًا.

4. **التخصيص وضبط سلوك الجلسة**: استخدم معرفات المستخدمين، المستأجرين، ومجموعات البيانات لتوفير رؤية مخصصة لكل شخص أو فريق على محرك الذاكرة المشترك.

5. **التوسع إلى وكلاء أكثر تعقيدًا**: قم بتوصيل Cognee بأطر عمل الوكلاء لبناء أنظمة متعددة الوكلاء تشترك جميعها في نفس طبقة الذاكرة. *إطار عمل Microsoft Agent x مكون إضافي لـ Cognee قادم قريبًا.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالترجمة البشرية الاحترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
